# Advanced Topics

<div class="alert alert-block alert-info"> 
<h2>Overview</h2>
This notebook will cover basic multi-threading and programming Nvidia GPUs in Julia.

Check out the [JuliaGPU](https://juliagpu.org/) orginization page for more on how to program GPUs in Julia as well as how to program GPUs from other vendors like AMD and Intel.


<strong>Objectives:</strong>
* How to parallelize for loops with `Threads.@threads`.
* Demonstrate the high-level interface for programming GPUs in Julia.

</div>


In [3]:
using BenchmarkTools
using OhMyThreads

### Multi-Threading
Julia's base [Threads](https://docs.julialang.org/en/v1/manual/multi-threading/) library provides the `Threads.@threads` macro to turn any for loop into a parallel for loop. To change the number of threads Julia has access to we can define the environment variable `JULIA_NUM_THREADS`. This must be done before starting the notebook (e.g. `export JULIA_NUM_THREADS=4`).

In [ ]:
println("Julia has access to $(Threads.nthreads()) threads")
print(ENV["JULIA_NUM_THREADS"])

In [ ]:
function single_thread!(vals)
    for i in eachindex(vals)
        vals[i] = sqrt(vals[i])*sqrt(vals[i])
    end
    return vals
end

function all_threads!(vals)
    Threads.@threads :static for i in eachindex(vals)
        vals[i] = sqrt(vals[i])*sqrt(vals[i])
    end
    return vals
end

function broadcasted!(vals)
    vals .= sqrt.(vals).*sqrt.(vals)
    return vals
end

In [7]:
vals = abs.(rand(1000000));

In [ ]:
@benchmark single_thread!($vals)

In [ ]:
@benchmark broadcasted!($vals)

In [ ]:
@benchmark all_threads!($vals)

The Threads library does not let you set the number of tasks, to do that we can use the `OhMyThreads.jl` library. In `OhMyThreads` the `Threads.@threads` macro is replaced with the `@tasks` macro inside of which we can set certain properties like `ntasks` as well as the type of thread scheduling.

In [ ]:
function ohmythreads!(vals, n_threads = Threads.nthreads())
    @tasks for i in eachindex(vals)
        @set ntasks = n_threads
        @set scheduler = :static
        vals[i] = sqrt(vals[i])*sqrt(vals[i])
    end
    return vals
end

In [ ]:
@benchmark ohmythreads!(vals)

### GPU Array Programming
With the power of dynamic dispacth, we can perform many operations on the GPU just by changing the type of our data from `Array` to `CuArray` (or the analagous type for AMD, Intel, or Apple). The packge environment does not have access to a GPU so the test the cells below install the `CUDA.jl` package and try running the code yourself!

```julia
# This example is for CUDA but is the same principle for all the GPU libraries
#using CUDA or AMDGPU or oneAPI or Metal or KernelAbstractions
using CUDA

# Moving data from CPU to GPU.
A = rand(3,3)
cu_B = CuArray{Float32}(A)

# Can also pre-allocate storage on the GPU and copy data to it.
gpu_storage = CUDA.zeros(Float32, 3, 3)
gpu_array2 = copyto!(cu_B, A)
```

If we multiply `cu_B` by itself, this multiplcation will take place on the GPU automatically because the type of `cu_B` is a `CuArray`!

```julia
cu_C = cu_B * cu_B 
```

There are many more high-level functions that "just work" on the GPU. For example, all unary operations like `+`,`-`,`*` have GPU implementations. There are also abstractions such as `reduce` and `mapreduce` which allow you to apply an operation or function across an array. Note that all high-level GPU operations should act on an entire array and not a single element at a time.

```julia
cu_C = CUDA.rand(5)
sum_of_C = sum(cu_C)
sum_of_C_also = reduce(+, cu_C)

f = (x) -> x^2
C_squared = map(f, cu_C)
sum_of_C_squared = mapreduce(f, +, cu_C)
```